In [ ]:
import pandas as pd
import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import lightgbm as lgbm
import numpy as np
from sklearn.model_selection import KFold
import os
from sklearn.feature_extraction.text import CountVectorizer 
import sklearn.decomposition as skld
from tqdm import tqdm_notebook as tqdm

In [ ]:
def create_lists(path):
    files = glob.glob('../data/news/'+path+'/*.txt')   
    id_articles = []
    articles_contents = []
    for file in files:
        id_article = (file.split("\\")[1]).split('.')[0];
        f = open('../data/news/'+path+'/'+id_article+'.txt','r', encoding='utf-8').read()
        id_articles.append(int(id_article))
        articles_contents.append(f)
    return id_articles, articles_contents

X_train_ids, X_train_contents = create_lists('training')
X_test_ids, X_test_contents = create_lists('test')


In [ ]:
def get_Y_train():
    true_val = []
    indexes = []
    articles = open('../data/labels_training.txt','r').readlines()[1:]
    for i in articles:
        true_val.append(int(i.split(',')[1][:-1]))
        indexes.append(i.split(',')[0])
    return pd.DataFrame(true_val,columns=['fake_news'],index=indexes)

Y_train = get_Y_train().sort_index()

In [ ]:
def sentense2cleanTokens(sent):
    sent = sent.lower()
    sent = "".join([x if x.isalpha() else " " for x in sent])
    sent = " ".join(sent.split())
    return sent

In [ ]:
X_train = []
for i in X_train_contents:
    X_train.append(sentense2cleanTokens(i))
    
X_test = []
for j in X_test_contents:
    X_test.append(sentense2cleanTokens(j))
    
Y_train = list(Y_train['fake_news'])

In [ ]:
#Peut etre a remettre 10 en min_df !
vec = TfidfVectorizer(max_df=0.7, min_df=0.1, max_features=100000, stop_words='english')
#vec = CountVectorizer(max_df=0.7, min_df=10, max_features=100000, stop_words='english') 
X = vec.fit_transform(X_train + X_test)
df = pd.DataFrame(X.toarray(), index=X_train_ids + X_test_ids, columns=vec.get_feature_names())


<u><b>Ajout plus grande phrase, nombre de phrases</b></u>

In [ ]:
plus_grande_phrase = []
number_sentences = []
for i in X_train_contents:
    maxi_phrase = 0
    nb_sent = 0
    for sentence in i.split("."):
        nb_sent += 1
        if len(sentence) > maxi_phrase:
            maxi_phrase = len(sentence)
    plus_grande_phrase.append(maxi_phrase)
    number_sentences.append(nb_sent)
for i in X_test_contents:
    maxi_phrase = 0
    nb_sent = 0
    for sentence in i.split("."):
        nb_sent += 1
        if len(sentence) > maxi_phrase:
            maxi_phrase = len(sentence)
    plus_grande_phrase.append(maxi_phrase)
    number_sentences.append(nb_sent)

df['plus_grande_phrase']= plus_grande_phrase
df['nb_sent'] = number_sentences

##### partie william

In [ ]:
longueur_article = []
for article in X_train:
    longueur_article.append(len(article))
for article in X_test:
    longueur_article.append(len(article))
df['len_article'] = longueur_article

In [ ]:
longueur_titre = []
for article in X_train_contents:
    longueur_titre.append(len(article.split('\n')[0]))
for article in X_test_contents:
    longueur_titre.append(len(article.split('\n')[0]))
df['len_title'] = longueur_titre


In [ ]:
nb_word = []
nb_entity = []
for article in X_train_contents:
    cpt = 0
    for word in article.split(' '):
        if word[:1].isupper() or word.isupper():
             cpt += 1
    nb_word.append(len(article.split(' ')))
    nb_entity.append(cpt)
for article in X_test_contents:
    cpt = 0
    for word in article.split(' '):
        if word[:1].isupper() or word.isupper():
             cpt += 1
    nb_word.append(len(article.split(' ')))
    nb_entity.append(cpt)
    
df['nb_word'] = nb_word
#C'est plutôt le nombre de mots commençant par une majuscule en fait
df['nb_entity'] = nb_entity

<b> Partie test de truc noul </b>

In [ ]:
last_sent = []
for article in X_train_contents:
    last_sent.append(len(article.split('\n')[-1]))
for article in X_test_contents:
    last_sent.append(len(article.split('\n')[-1]))
df['last_sent'] = last_sent

<u><b>Ajout en fonction du nombre de type de mot</b></u>

In [ ]:
"""from nltk import ne_chunk, pos_tag, word_tokenize
nb_nnp = []
for article in X_train:
    cpt = 0
    for i in pos_tag(word_tokenize(article)):
        if i[1] == 'NNP':
            cpt += 1
    nb_nnp.append(cpt)
for article in X_test:
    cpt = 0
    for i in pos_tag(word_tokenize(article)):
        if i[1] == 'NNP':
            cpt += 1
    nb_nnp.append(cpt)
df['nb_nnp'] = nb_nnp"""

##Pas pertinent

<b><u>Ajout des infos users</u></b>

In [ ]:
newsUsers = open('../data/newsUser.txt','r').readlines()

In [ ]:
id_articles = []
id_users = []
nb_propa = []
for i in newsUsers:
    id_articles.append(str(i.split("\t")[0]))
    id_users.append(str(i.split("\t")[1]))
    nb_propa.append(int((i.split("\t")[2]).split("\n")[0]))

In [ ]:
article_users = {}
for i in range(len(id_articles)):
    users_propa = {}
    if id_articles[i] in article_users.keys():
        users_propa = article_users[id_articles[i]]
    users_propa[id_users[i]] = nb_propa[i]
    article_users[id_articles[i]] = users_propa

<b><i>article_users : dico de la forme clé id_article : dico associé clé id_user values nb de propagation de cet article par cet user

In [ ]:
UsersUsers = open('../data/UserUser.txt','r').readlines()

In [ ]:
id_users_followers = []
id_users_followed = []
for i in UsersUsers:
    id_users_followers.append(str(i.split("\t")[0]))
    id_users_followed.append(str(i.split("\t")[1]).split("\n")[0])

In [ ]:
user_user = {}
for i in range(len(id_users_followers)):
    user_followed = []
    if id_users_followers[i] in user_user.keys():
        user_followed = user_user[id_users_followers[i]]
    user_followed.append(id_users_followed[i])
    user_user[id_users_followers[i]] = user_followed

<b><i> user_user : dico de la forme clé du dico suit tous les utilisateurs de la liste associé

In [ ]:
nb_propa_article = []
for i in df.index:
    nb_propa_article.append(sum(article_users[str(i)].values()))
df['nb_propa_article'] = nb_propa_article

<b><u>Ajout des topics des articles </u></b> 
# TO DO

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
km = MiniBatchKMeans(n_clusters=len(df))
km.fit(X)

In [ ]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vec.get_feature_names()
for i in range(len(df)):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()s

<b><u>Partie Réseau</u></b>
# TO DO

In [ ]:
df.head()

##### Séparation de notre dataframe en deux df : train / test

In [ ]:
df_train = pd.DataFrame(df.loc[X_train_ids])
df_test = pd.DataFrame(df.loc[X_test_ids])

<u><b>Partie entraînement du modèle</b></u>

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB

In [ ]:
#### Contribution de nos features

X = df_train 
y = Y_train    


model = GradientBoostingClassifier(loss = 'exponential')
model.fit(X,y)

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='barh')
plt.show()

In [ ]:
model = GradientBoostingClassifier(loss = 'exponential')
pred = np.mean(cross_val_score(model, df_train, Y_train, cv=10))
print(pred)
clf_gbc = GradientBoostingClassifier(loss = 'exponential')
clf_gbc.fit(df_train, Y_train)

In [ ]:
#### Contribution de nos features

X = df_train 
y = Y_train    


model = RandomForestClassifier(n_estimators= 1400, min_samples_split= 2, min_samples_leaf= 1, max_features='auto', max_depth= 40, bootstrap= False)
model.fit(X,y)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(30).plot(kind='barh')
plt.show()

In [ ]:
model = RandomForestClassifier(n_estimators= 1400, min_samples_split= 2, min_samples_leaf= 1, max_features='auto', max_depth= 40, bootstrap= False)
pred = np.mean(cross_val_score(model, df_train, Y_train, cv=10))
print(pred)
clf_gbc = RandomForestClassifier(n_estimators= 1400, min_samples_split= 2, min_samples_leaf= 1, max_features='auto', max_depth= 40, bootstrap= False)
clf_gbc.fit(df_train, Y_train)

<b><u>Pour push sur Kaggle</u></b>

In [ ]:
pred_test = clf_gbc.predict(df_test)

In [ ]:
os.remove("../pred_test.txt")
fichier = open("../pred_test.txt", "a")
fichier.write("doc,class")
for i in range(len(pred_test)):
    fichier.write("\n" + str(X_test_ids[i]) + ',' + str(pred_test[i]))
fichier.close()

In [ ]:
#Calcul du score

In [ ]:
fichier_100 = open("../pred_test_v4.txt", "r").readlines()[1:]
fichier_push = open("../pred_test.txt", "r").readlines()[1:]

In [ ]:
result_a_avoir = []
result_eu = []
for i in fichier_100:
    result_a_avoir.append(i.split(',')[1])
for i in fichier_push:
    result_eu.append(i.split(',')[1])


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print(accuracy_score(result_eu, result_a_avoir))